# Detect (with the) Model
## Step 1 - run the TENSORFLOW model locally

This example uses TensorFlow saved_model.pb.   Which begs the question, what is the difference between a saved_model.pb and a frozen_inference_graph.pb?  Well
https://stackoverflow.com/questions/52934795/what-is-difference-frozen-inference-graph-pb-and-saved-model-pb

ref:  
https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb  
https://blog.algorithmia.com/deep-dive-into-object-detection-with-open-images-using-tensorflow/  
https://pythonprogramming.net/tensorflow-object-detection-api-self-driving-car/  


In [49]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import json

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since we cloned tensorflow/models under code.
cwd = os.getcwd()
models = os.path.join(cwd, 'code/models/research/')
slim = os.path.join(cwd, 'code/models/research/slim')
sys.path.append(models)
sys.path.append(slim)

from object_detection.utils import ops as utils_ops

In [2]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## Global Environment Variables

We are using objects and scripts in the project as much as possible.   They require environment variables to pass along where stuff is stored

### Where did the model come from?
When you trained (Step 3 - SageMaker HOSTED), 
- The eval_spec (inside your training program) includes an Exporter
  - the Exporter will export the model to export/Server/<some timestamp> creating:
    - saved_graph.pb
    - variables
- SageMaker included this in the output/model.tar.gz that it sent to S3
- at the end of the Step 3 notebook, you retrieved the tarball and extracted it
  - so now it is in trained_model/export/Servo/(you may have serveral here if your ran repeatedly)
    
    

In [11]:
PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/new_jpeg_images")

MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1564865938")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/unverified_annotations")

S3_TEST_IMAGES = "s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/test_images/"
SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/new_jpeg_images/20190710_variety_1562781002.jpg"

# you can get data using the TrainModel_Step1_Local notebook
TEST_TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "code/tfrecords/test/")

In [4]:
# This is needed to display the images.
%matplotlib inline

## TensorFlow Serving

https://www.tensorflow.org/tfx/serving/serving_basic

SageMaker uses TensorFlow Serving - the advantage is the automation associated with directing traffic between models and upgrading.   Most of this is transparent to you.

### Exploring the Model
This utility is part of tensorflow (models?).  It will tell you about the input & output tensors

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/saved_model_cli.py


In [5]:
! ls {MODEL_PATH}
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve 

saved_model.pb	variables
The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "serving_default"
SignatureDef key: "tensorflow/serving/predict"


In [6]:
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve --signature_def tensorflow/serving/predict

The given SavedModel SignatureDef contains the following input(s):
  inputs['serialized_example'] tensor_info:
      dtype: DT_STRING
      shape: ()
      name: tf_example:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 4)
      name: detection_boxes:0
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: detection_classes:0
  outputs['detection_multiclass_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 12)
      name: detection_multiclass_scores:0
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: detection_scores:0
  outputs['num_detections'] tensor_info:
      dtype: DT_FLOAT
      shape: (1)
      name: num_detections:0
  outputs['raw_detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 1917, 4)
      name: raw_detection_boxes:0
  outputs['raw_dete

In [7]:
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['serialized_example'] tensor_info:
      dtype: DT_STRING
      shape: ()
      name: tf_example:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 4)
      name: detection_boxes:0
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: detection_classes:0
  outputs['detection_multiclass_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 12)
      name: detection_multiclass_scores:0
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: detection_scores:0
  outputs['num_detections'] tensor_info:
      dtype: DT_FLOAT
      shape: (1)
      name: num_detections:0
  outputs['raw_detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 1917, 4)
      name: raw_detection_boxes:0
  outputs['raw_dete

Evidently the Exporter uses something called a SavedModel.simple  

You need the tags, ref:  https://github.com/tensorflow/tensorflow/issues/25288  
The stackoverflow reference above had [].  Without them - i.e. [] - you'll get an error 
  
#### Here you need a saved model Graph

In [ ]:
# alternative 1
loaded_model = tf.saved_model.load(sess=tf.Session(), tags=[tf.saved_model.tag_constants.SERVING], export_dir=MODEL_PATH)

In [8]:
# alternative 2
# this is what is in the tutorial:
# https://www.tensorflow.org/guide/saved_model
! ls {MODEL_PATH}
with tf.Session() as sess:
    detection_graph = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], MODEL_PATH)

saved_model.pb	variables


W0824 13:42:22.694341 140046763558720 deprecation.py:323] From <ipython-input-8-b36c4d5cb2ce>:6: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0824 13:42:23.811750 140046763558720 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1282: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [9]:
predict = loaded_model.signature_def["tf_example"]
print (type(predict))


NameError: name 'loaded_model' is not defined

## SavedModel

### Understanding the MetaGraphDef

https://www.tensorflow.org/tfx/serving/signature_defs

Pay attention to versions.   Old: Signatures New: SignatureDefs   Much of the documentation I'm reading is out of date (and frustrating because it won't work!)


In [ ]:
# what are the model signature keys?
print ("loaded model:", type(loaded_model))   # don't attempt to print loaded_model -- too big?
# print ("Signature Keys:", list(loaded_model.signatures.keys()))

In [ ]:
import tensorflow.core.protobuf.meta_graph_pb2

sde = loaded_model.SignatureDefEntry
print (type(sde))
#sde['tensorflow/serving/predict']
print (sde)


In [ ]:
l = loaded_model.SignatureDefEntry
print (type(l), l)

### Load Label Map

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP, use_display_name=True)

## Retreive Data
Copy the test images locally

In [ ]:
! aws s3 cp {S3_TEST_IMAGES} {IMAGE_DIR} --recursive

In [ ]:
# Helper Function
# load an image and resturn a numpy array

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
dir_list = os.listdir(IMAGE_DIR)
image_list = list()
for f in dir_list:
    full_path = os.path.join(IMAGE_DIR, f)
    if os.path.isfile(full_path) and os.path.splitext(f)[1].lower() == '.jpg':
        image_list.append(full_path)

# limitations with the way we are displaying
image_list = image_list[:1]
print ("Image Count:", len(image_list))
print ("Sample:",image_list[0])

In [ ]:
# Size, in inches, of the output images.
IMAGE_SIZE = (6, 4)

### Understanding the Model

TensorFlow provides the framework for the model - but you really don't know how the model was programmed for input & output.    You could dissect the MobileNet source code but the easier method in this case is to examine a sample (that came from tensorflow/models/object_detection)

#### How do you pass data into the Model?
The model has 3400+ operations and a lot of tensors.  
Image Tensor: Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8)  

##### Input Tensor

##### Output Tensors
We can are looking for 4 tensors (and they are float32)  
What is a detection mask?  

#### How do you process the inference?

In [ ]:
# NAME - get this from the console
ENDPOINT_NAME = "ep-mobilenet-ssd"

In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(ENDPOINT_NAME, sagemaker_session)

print (type(predictor))

In [ ]:
# get attributes from the SAVED GRAPH  
# - not the frozen graph.pb
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      print ("ALL model operations:", type(ops), len(ops), ops)
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      print ("tensor names:", type(all_tensor_names), len(all_tensor_names))
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_value = tf.get_default_graph().get_tensor_by_name(tensor_name)
          print (tensor_name, tensor_value)
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
      if 'detection_masks' in tensor_dict:
        print ("*** detection mask in the tensor dict ***")
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      print ("image_tensor:", image_tensor)
        
      # Run inference
      print ("tensor dict:", tensor_dict)
      output_dict = sess.run(tensor_dict,feed_dict={image_tensor: image})
      # output_dict = predictor.predict({'image_tensor': image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
for image_path in image_list:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

### Model CLI - run

```
saved_model_cli run [-h] --dir DIR --tag_set TAG_SET --signature_def
                           SIGNATURE_DEF_KEY [--inputs INPUTS]
                           [--input_exprs INPUT_EXPRS]
                           [--input_examples INPUT_EXAMPLES] [--outdir OUTDIR]
                           [--overwrite] [--tf_debug]
                           ```

In [ ]:
saved_model_cli run [-h] --dir DIR --tag_set TAG_SET --signature_def
                           SIGNATURE_DEF_KEY [--inputs INPUTS]
                           [--input_exprs INPUT_EXPRS]
                           [--input_examples INPUT_EXAMPLES] [--outdir OUTDIR]
                           [--overwrite] [--tf_debug]

### Input Examples 
Not an example of input - input = tf.train.Example  

So, how to create Example from a tfrecord

In [14]:
# This is needed since we cloned tensorflow/models under code.
cwd = os.getcwd()
models = os.path.join(cwd, 'code/models/research/')
slim = os.path.join(cwd, 'code/models/research/slim')
sys.path.append(models)
sys.path.append(slim)

from object_detection.inference import detection_inference

# cfa_utils
#  - some feature helper functions
#  - a Feature - (common format) for object detection
from code.cfa_utils.example_utils import feature_obj_detect
from code.cfa_utils.example_utils import int64_feature, int64_list_feature, bytes_feature, bytes_list_feature, float_list_feature


! ls {TEST_TFRECORDS_PATH}
input_tfrecord_paths = [TEST_TFRECORDS_PATH]


serialized_example_tensor, image_tensor = detection_inference.build_input( input_tfrecord_paths)

print ("serialized_example_tensor", serialized_example_tensor)
print ("image_tensor", image_tensor.shape)

serialized_example_tensor Tensor("ReaderReadV2_2:1", shape=(), dtype=string)
image_tensor (1, ?, ?, 3)


In [50]:
# feature dict was defined on the fly
img_path = os.path.join(SAMPLE_IMAGE)
with tf.io.gfile.GFile(img_path, 'rb') as fid:
   encoded_jpg = fid.read()

type(encoded_jpg)

tf_example = tf.train.Example(features=tf.train.Features(feature={'image/encoded': bytes_feature(encoded_jpg)}))
print ("tf.train.Example:", type(tf_example))
# print ("tf.train.Example Value:", tf_example)
ex_list = [tf_example]
print (type(ex_list), type(ex_list[0]))
# = {'image/encoded': bytes_feature(encoded_jpg)}
d = {'image/encoded' : bytes_feature(encoded_jpg)}
# print (type(d), d)
# INPUT_EXAMPLES = 'serialized_example=[' + d + ']'
d_str = json.dumps(d)
print (d_str)

tf.train.Example: <class 'tensorflow.core.example.example_pb2.Example'>
<class 'list'> <class 'tensorflow.core.example.example_pb2.Example'>


TypeError: Object of type 'Feature' is not JSON serializable

In [39]:
! saved_model_cli run --dir {MODEL_PATH} --tag_set serve --signature_def tensorflow/serving/predict --input_examples serialized_example=ex_list

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/saved_model_cli", line 11, in <module>
    sys.exit(main())
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 639, in run
    args.inputs, args.input_exprs, args.input_examples)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 551, in load_inputs_from_input_arg_string
    input_examples = preprocess_input_examples_arg_string(input_examples_str)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 464, in preprocess_input_examples_arg_string
    input_dict = preprocess_input_exprs_arg_string(input_e

## Conclusion

If you want the annotations - you'll find the annotations tarball in data/ directory
use the Notebook browser to download it

The fastest, easiest way to review (and correct / verify) is to use labelImg program which will merge the image and annotation

The main conclusion here is our model works.

In [ ]:
print (detection_graph)

In [ ]:
image = Image.open(SAMPLE_IMAGE)
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)

print (image_np_expanded.shape)

In [ ]:
image = image_np_expanded

with detection_graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      print ("ALL model operations:", type(ops), len(ops))
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      print ("tensor names:", type(all_tensor_names), len(all_tensor_names))
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_value = tf.get_default_graph().get_tensor_by_name(tensor_name)
          print (tensor_name, tensor_value)
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
      if 'detection_masks' in tensor_dict:
        print ("*** detection mask in the tensor dict ***")
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      print ("image_tensor:", image_tensor)
        
      # Run inference
      print ("tensor dict:", tensor_dict)
        
      # input into the model must be JSON
      output_dict = sess.run(tensor_dict,feed_dict={image_tensor: image})
      # output_dict = predictor.predict(tensor_dict,feed_dict={image_tensor: image})  # doesn't like feed_dict
      # output_dict = predictor.predict(tensor_dict, {image_tensor: image})
      # output_dict = predictor.predict(tensor_dict, feed_dict={"image_tensor": image_tensor})

      # ops includes:
      # <tf.Operation 'image_tensor' type=Placeholder>

In [ ]:
print (type(image_np_expanded), image_np_expanded.shape)

In [ ]:
output = predict(tf.constant(image_np_expanded))[]

In [ ]:

img = tf.keras.preprocessing.image.load_img(SAMPLE_IMAGE, target_size=[300, 300])
plt.imshow(img)
plt.axis('off')

In [ ]:
x = tf.keras.preprocessing.image.img_to_array(img)
print (type(x), x.shape)

x = tf.keras.applications.mobilenet.preprocess_input(x[tf.newaxis,...])
print (type(x), x.shape)

In [ ]:
def pred(x):
    example = tf.train.Example()
    example.features.feature["x"].float_list.value.extend([x])
    return loaded_model.signatures["predict"](examples=tf.constant([example.SerializeToString()]))

In [ ]:
pred(x)